In [ ]:
# papermill parameters
zonas = [31, 37, 39, 55]
ruta = "/Volumes/Juanjo_4TB/Disco_4T/Master_Inteligencia_Artificial/Curso_2024_25/01_Python_para_Inteligencia_Artificial/Estudio/TFM"


### Importa las librerias

In [12]:

import sys
import os

# Añadir la carpeta raíz del proyecto al sys.path
sys.path.append(os.path.abspath(".."))

import importlib
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import src.utilidades as uti
import src.graficos as gra
import src.parametros_sismicos as par

from shapely.geometry import Point

importlib.reload(uti)
importlib.reload(par)

<module 'src.parametros_sismicos' from '/Volumes/Juanjo_4TB/Disco_4T/Master_Inteligencia_Artificial/Curso_2024_25/01_Python_para_Inteligencia_Artificial/Estudio/TFM/src/parametros_sismicos.py'>

### Lectura del catálogo de terremotos (*.csv)

In [13]:
# Leer CSV
filename = "Catalogo_IGN_1900_2025_todo.csv"
output_file=ruta+"/data/catalogo/"+filename

df = pd.read_csv(output_file, sep=";", encoding="utf-8", low_memory=False)

# Limpiar nombres de columnas: quitar espacios y caracteres raros
df.columns = df.columns.str.strip()              # quita espacios a izquierda y derecha
df.columns = df.columns.str.replace(r"\s+", "_", regex=True)  # cambia espacios internos por "_"
df.columns = df.columns.str.replace(r"[^\w]", "", regex=True) # elimina caracteres no alfanuméricos

# Substituye la columna Localizacin por Localizacion
if "Localizacin" in df.columns:
    df = df.rename(columns={"Localizacin": "Localizacion"})

# Normaliza las columnas del DataFrame
df=uti.normalizar_columnas(df)


print(df.columns.tolist())

df.info()
df.head()

['Evento', 'Fecha', 'Hora', 'Latitud', 'Longitud', 'Prof_Km', 'Inten', 'Mag', 'Tipo_Mag', 'Localizacion']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 141439 entries, 0 to 141438
Data columns (total 10 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Evento        141439 non-null  object 
 1   Fecha         141439 non-null  object 
 2   Hora          141439 non-null  object 
 3   Latitud       141439 non-null  float64
 4   Longitud      141439 non-null  float64
 5   Prof_Km       141439 non-null  float64
 6   Inten         141439 non-null  object 
 7   Mag           141439 non-null  object 
 8   Tipo_Mag      141439 non-null  object 
 9   Localizacion  141439 non-null  object 
dtypes: float64(3), object(7)
memory usage: 10.8+ MB


,Evento,Fecha,Hora,Latitud,Longitud,Prof_Km,Inten,Mag,Tipo_Mag,Localizacion
0,2494,16/02/1900,05:00:00,43.0,0.1500,0.0,III,,,Bagneres-de-Bigorre.FRA
1,2495,04/03/1900,09:00:00,40.0,3.8333,0.0,IV,,,Ciutadella.IB
2,2496,09/03/1900,04:20:00,37.6,-1.3167,0.0,IV,,,Mazarrón.MU
3,2497,25/04/1900,18:00:00,35.7,-0.6000,0.0,VI,,,ORAN.ARG
4,2498,09/06/1900,23:50:00,42.6,-5.5667,0.0,III,,,León


### Unifica la Fecha y Hora en una columna

In [14]:
# Unificar fecha y hora en un datetime
df = df.assign(FechaHora=pd.to_datetime(df["Fecha"] + " " + df["Hora"], dayfirst=True, errors="coerce")) \
       .drop(columns=["Fecha", "Hora"])

df.head()

,Evento,Latitud,Longitud,Prof_Km,Inten,Mag,Tipo_Mag,Localizacion,FechaHora
0,2494,43.0,0.1500,0.0,III,,,Bagneres-de-Bigorre.FRA,1900-02-16 05:00:00
1,2495,40.0,3.8333,0.0,IV,,,Ciutadella.IB,1900-03-04 09:00:00
2,2496,37.6,-1.3167,0.0,IV,,,Mazarrón.MU,1900-03-09 04:20:00
3,2497,35.7,-0.6000,0.0,VI,,,ORAN.ARG,1900-04-25 18:00:00
4,2498,42.6,-5.5667,0.0,III,,,León,1900-06-09 23:50:00


### Normalizas las columnas Tipo_Mag y Mag y las convierte en numérica

In [16]:
# Quitar espacios, convertir a numérico, forzar enteros si aplica
df["Tipo_Mag"] = (
    df["Tipo_Mag"]
    .astype(str)        # asegurar que todo es string
    .str.strip()        # quitar espacios en blanco
    .replace("", None)  # convertir cadenas vacías en NaN
)

# Convertir a número, los errores se vuelven NaN
df["Tipo_Mag"] = pd.to_numeric(df["Tipo_Mag"], errors="coerce").astype("Int64")

# Convertir a número, los errores se vuelven NaN
df["Mag"] = pd.to_numeric(df["Mag"], errors="coerce")


df.head()

,Evento,Latitud,Longitud,Prof_Km,Inten,Mag,Tipo_Mag,Localizacion,FechaHora
0,2494,43.0,0.1500,0.0,III,NaN,<NA>,Bagneres-de-Bigorre.FRA,1900-02-16 05:00:00
1,2495,40.0,3.8333,0.0,IV,NaN,<NA>,Ciutadella.IB,1900-03-04 09:00:00
2,2496,37.6,-1.3167,0.0,IV,NaN,<NA>,Mazarrón.MU,1900-03-09 04:20:00
3,2497,35.7,-0.6000,0.0,VI,NaN,<NA>,ORAN.ARG,1900-04-25 18:00:00
4,2498,42.6,-5.5667,0.0,III,NaN,<NA>,León,1900-06-09 23:50:00


### Recorta el catálogo de datos desde 2002 en adelante

In [17]:

# Anterior al 2002, el número de terremotos por año registrados es muy bajo

# Definir fecha/hora de corte
fecha_corte = pd.to_datetime("2002-01-01 00:00:00")

# Filtrar
df_2002 = df[df["FechaHora"] > fecha_corte]

df_2002.head()

,Evento,Latitud,Longitud,Prof_Km,Inten,Mag,Tipo_Mag,Localizacion,FechaHora
20265,300074,37.2294,-3.6999,2.6,,1.6,2,NW ATARFE.GR,2002-01-01 02:34:47
20266,300124,37.0589,-3.6394,9.8,,2.4,2,S OTURA.GR,2002-01-01 12:43:10
20267,300131,37.7699,-4.8811,10.3,,2.8,2,E GUADALCÁZAR.CO,2002-01-01 13:44:32
20268,387003,41.8125,-8.5080,0.0,,2.4,2,NE PONTE DE LIMA.POR,2002-01-01 15:05:05
20269,300218,35.2614,-4.6484,26.3,,2.8,2,NE JEBHA.MAC,2002-01-01 15:12:06


### Filtra Tipo_Mag = 2, 3, 4, 5 y 6, y convierte todos al tipo 4 (mbLgL)

In [18]:
df_out = uti.homogenizar_dataframe(df_2002, col_tipo="Tipo_Mag", col_mag="Mag", col_fecha="FechaHora")

print(df_out[["Tipo_Mag", "Mag", "FechaHora", "Mag_mbLgL"]].head())

[REPORTE] Total=121174, OK=121172, NaN=2 (0.00%)
       Tipo_Mag  Mag           FechaHora  Mag_mbLgL
20265         2  1.6 2002-01-01 02:34:47   0.123673
20266         2  2.4 2002-01-01 12:43:10   1.121224
20267         2  2.8 2002-01-01 13:44:32   1.620000
20268         2  2.4 2002-01-01 15:05:05   1.121224
20269         2  2.8 2002-01-01 15:12:06   1.620000


### Crea un GeoDataFrame a partir del Dataframe

In [19]:
# Crear GeoDataFrame de epicentros 
geometry = [Point(xy) for xy in zip(df_out["Longitud"], df_out["Latitud"])]
gdf_2002= gpd.GeoDataFrame(df_out, geometry=geometry, crs="EPSG:4326")  # WGS84 (lat/lon)

gdf_2002.head()

,Evento,Latitud,Longitud,Prof_Km,Inten,Mag,Tipo_Mag,Localizacion,FechaHora,Mag_mbLgL,geometry
20265,300074,37.2294,-3.6999,2.6,,1.6,2,NW ATARFE.GR,2002-01-01 02:34:47,0.123673,POINT (-3.6999 37.2294)
20266,300124,37.0589,-3.6394,9.8,,2.4,2,S OTURA.GR,2002-01-01 12:43:10,1.121224,POINT (-3.6394 37.0589)
20267,300131,37.7699,-4.8811,10.3,,2.8,2,E GUADALCÁZAR.CO,2002-01-01 13:44:32,1.620000,POINT (-4.8811 37.7699)
20268,387003,41.8125,-8.5080,0.0,,2.4,2,NE PONTE DE LIMA.POR,2002-01-01 15:05:05,1.121224,POINT (-8.508 41.8125)
20269,300218,35.2614,-4.6484,26.3,,2.8,2,NE JEBHA.MAC,2002-01-01 15:12:06,1.620000,POINT (-4.6484 35.2614)


### Asigna la zona sismogénica (Zona_ID)

In [20]:

filename = "ZESIS_20150421c.shp"
output_file=ruta+"/data/zonas/"+filename

# Supongamos que ya tienes tu gdf con puntos (lat/lon)
gdf_2002 = par.asignar_zona_sismogenica(
    gdf_2002,
    shp_path=output_file,
    col_geom="geometry",
    col_id="ID"  
)

gdf_2002.head()


,Evento,Latitud,Longitud,Prof_Km,Inten,Mag,Tipo_Mag,Localizacion,FechaHora,Mag_mbLgL,geometry,Zona_ID
20265,300074,37.2294,-3.6999,2.6,,1.6,2,NW ATARFE.GR,2002-01-01 02:34:47,0.123673,POINT (-3.6999 37.2294),35.0
20266,300124,37.0589,-3.6394,9.8,,2.4,2,S OTURA.GR,2002-01-01 12:43:10,1.121224,POINT (-3.6394 37.0589),35.0
20267,300131,37.7699,-4.8811,10.3,,2.8,2,E GUADALCÁZAR.CO,2002-01-01 13:44:32,1.620000,POINT (-4.8811 37.7699),29.0
20268,387003,41.8125,-8.5080,0.0,,2.4,2,NE PONTE DE LIMA.POR,2002-01-01 15:05:05,1.121224,POINT (-8.508 41.8125),2.0
20269,300218,35.2614,-4.6484,26.3,,2.8,2,NE JEBHA.MAC,2002-01-01 15:12:06,1.620000,POINT (-4.6484 35.2614),42.0


### Filtra y guarda por zona sismogénica (Zona_ID)

In [29]:

for zona in zonas:
    df_zona = gdf_2002[gdf_2002["Zona_ID"] == zona].copy()

    # Elimina la fila si algún valor de "Mag_mbLgL" es NaN
    df_zona = df_zona.dropna(subset=["Mag_mbLgL"])


    filename = f"datos_zona_{zona}.pkl"
    output_file=ruta+"/pkl_files/"+filename
    
    
    df_zona.to_pickle(output_file)
    print(f"Guardado {filename} con {len(df_zona)} filas")



Guardado datos_zona_31.pkl con 1930 filas
Guardado datos_zona_37.pkl con 4460 filas
Guardado datos_zona_39.pkl con 485 filas
Guardado datos_zona_55.pkl con 3574 filas
